In [2]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models, layers, optimizers
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import backend as K
import pandas as pd

from keras.layers.convolutional import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [5]:
base_dir = 'F:\\얼굴표정\\1.datasets_balance'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=10, class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=10, class_mode='categorical')

Found 656 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [6]:
conv_base = VGG16(weights='imagenet', include_top=False,
                  input_shape=(150, 150, 3))

#print(conv_base.summary())

W0630 15:01:49.795568 11844 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 15:01:49.833465 11844 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 15:01:49.843438 11844 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0630 15:01:49.917241 11844 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0630 15:01:50.429921 11844 deprecation_wrapper.py:119] From C:\Users\Gacho

In [7]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
model = models.Sequential()
model.add(conv_base)
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten(name='flatten'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
print(model.summary())
model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_acc', patience=5, mode='auto')
history = model.fit_generator(train_generator, steps_per_epoch=10, epochs=50, validaton_data=val_generator, validation_steps=10,
                             callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 16,812,610
Trainable params: 9,177,346
Non-trainable params: 7,635,264
_________________________________________________________________
None


NameError: name 'val_generator' is not defined

In [6]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=10)
# print(test_acc)
acc = round(test_acc * 100, 2)
print(acc)

# y_pred = model.predict_generator(test_generator, steps=10)

K.clear_session()

0.7600000023841857
76.0


In [7]:
model.save('F:\\얼굴표정\\2.model_save\\balance_face_model_' + str(acc) +'.h5')